# Analyse the code

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import os

## load data
Load the data with 0 for empty values

In [2]:
def load_excel_to_df(filename):
    """
    Load an Excel file into a pandas DataFrame and replace empty values with 0.

    Parameters:
    -----------
    filename : str
        Path to the Excel file (.xlsx)

    Returns:
    --------
    pandas.DataFrame
        DataFrame containing the Excel data with empty values replaced by 0
    """
    # Check if file exists
    if not os.path.exists(filename):
        raise FileNotFoundError(f"The file {filename} does not exist.")

    # Check if file is an Excel file
    if not filename.endswith('.xlsx'):
        raise ValueError(f"The file {filename} is not an Excel file (.xlsx).")

    try:
        # Read the Excel file
        df = pd.read_excel(filename)

        # Replace empty values (NaN) with 0
        df = df.fillna(0)

        return df

    except Exception as e:
        raise Exception(f"Error loading Excel file: {str(e)}")

df = load_excel_to_df('structured_data.xlsx')

In [3]:
df.describe()


,num_stackdepth3_logs,evts_1,expandEvts_1,pruneBacktrackEvts_1,backtrackEvts_1,strengthenEvts_1,maxStackDepth_1,evts_2,expandEvts_2,pruneBacktrackEvts_2,...,expandEvts_ratio_1,pruneBacktrackEvts_ratio_1,expandEvts_ratio_2,pruneBacktrackEvts_ratio_2,expandEvts_ratio_3,pruneBacktrackEvts_ratio_3,expandEvts_ratio_4,pruneBacktrackEvts_ratio_4,expandEvts_ratio_5,pruneBacktrackEvts_ratio_5
count,690.000000,690.0,690.0,690.0,690.0,690.0,690.0,6.900000e+02,6.900000e+02,6.900000e+02,...,690.0,690.0,690.000000,690.000000,690.000000,690.000000,690.000000,690.000000,690.000000,690.000000
mean,2.326087,4.0,4.0,0.0,0.0,0.0,3.0,1.342984e+09,6.714918e+08,6.669157e+08,...,1.0,0.0,0.506421,0.471248,0.251681,0.231418,0.134380,0.127155,0.059995,0.057876
std,1.488042,0.0,0.0,0.0,0.0,0.0,0.0,1.283705e+09,6.418524e+08,6.368330e+08,...,0.0,0.0,0.018576,0.060241,0.251226,0.233928,0.224039,0.213020,0.163525,0.157937
min,1.000000,4.0,4.0,0.0,0.0,0.0,3.0,1.100000e+01,6.000000e+00,2.000000e+00,...,1.0,0.0,0.493421,0.181818,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,4.0,4.0,0.0,0.0,0.0,3.0,1.017850e+04,5.089750e+03,4.982000e+03,...,1.0,0.0,0.500000,0.487590,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.000000,4.0,4.0,0.0,0.0,0.0,3.0,1.895630e+09,9.478150e+08,9.475620e+08,...,1.0,0.0,0.500000,0.498040,0.428571,0.248718,0.000000,0.000000,0.000000,0.000000
75%,4.000000,4.0,4.0,0.0,0.0,0.0,3.0,2.617559e+09,1.308780e+09,1.288496e+09,...,1.0,0.0,0.500000,0.499755,0.500000,0.490346,0.499356,0.360000,0.000000,0.000000
max,5.000000,4.0,4.0,0.0,0.0,0.0,3.0,3.013720e+09,1.506860e+09,1.480088e+09,...,1.0,0.0,0.636364,0.499999,0.615385,0.499999,0.588235,0.499999,0.534884,0.499999


In [4]:
def train_random_forest_model(df, include_ratios=True, save_plots=False, output_dir='plots'):
    """
    Trains two Random Forest models to predict final_expandEvts: one on censored=0 data,
    one on all data with censored as a feature. Outputs RMSE, MAPE, and feature importance.

    Parameters:
    - df (pandas.DataFrame): Input DataFrame with solver features, ratio features, and final_expandEvts.
    - include_ratios (bool): If True, includes ratio features; if False, excludes them.
    - save_plots (bool): If True, saves scatter and feature importance plots to output_dir.
    - output_dir (str): Directory to save plots (default: 'plots').

    Returns:
    - dict: Contains RMSE, MAPE, feature importance DataFrames, and test predictions for both models.
    """
    try:
        # Create a copy to avoid modifying the input DataFrame
        df = df.copy()
        
        # Ensure missing log features are imputed
        print("Imputing missing log features...")
        for i in range(1, 6):
            for feature in ['evts', 'expandEvts', 'pruneBacktrackEvts', 'backtrackEvts', 'strengthenEvts', 'maxStackDepth']:
                col = f'{feature}_{i}'
                if col in df:
                    df.loc[df['k'] == 3, col] = df.loc[df['k'] == 3, col].fillna(0)
                    df.loc[df['num_stackdepth3_logs'] < i, col] = df.loc[df['num_stackdepth3_logs'] < i, col].fillna(0)
        
        # Feature selection
        exclude_cols = ['filename', 'final_expandEvts', 'stop_iter', 'final_maxStackDepth']
        if not include_ratios:
            exclude_cols.extend([f'expandEvts_ratio_{i}' for i in range(1, 6)])
            exclude_cols.extend([f'pruneBacktrackEvts_ratio_{i}' for i in range(1, 6)])
        features = [col for col in df.columns if col not in exclude_cols]
        print("Selected features:", features)
        
        # Print target statistics for context
        print("\nTarget (final_expandEvts) statistics:")
        print(f"Mean (censored=0): {df[df['censored'] == 0]['final_expandEvts'].mean():.2f}")
        print(f"Std (censored=0): {df[df['censored'] == 0]['final_expandEvts'].std():.2f}")
        print(f"Mean (all data): {df['final_expandEvts'].mean():.2f}")
        print(f"Std (all data): {df['final_expandEvts'].std():.2f}")
        
        # Calculate MAPE
        def calculate_mape(y_true, y_pred):
            mask = y_true != 0  # Avoid division by zero
            return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100 if mask.sum() > 0 else np.nan
        
        # Model 1: Censored=0
        print("\nTraining Model 1 (censored=0)...")
        df_censored = df[df['censored'] == 0]
        if df_censored.empty:
            print("Warning: No censored=0 instances found. Skipping Model 1.")
            rmse_censored = None
            mape_censored = None
            importance_censored = None
            y_test_c = None
            y_pred_c = None
        else:
            X_censored = df_censored[features]
            y_censored = df_censored['final_expandEvts']
            X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X_censored, y_censored, test_size=0.2, random_state=42)
            
            rf_censored = RandomForestRegressor(n_estimators=100, random_state=42)
            rf_censored.fit(X_train_c, y_train_c)
            y_pred_c = rf_censored.predict(X_test_c)
            rmse_censored = np.sqrt(mean_squared_error(y_test_c, y_pred_c))
            mape_censored = calculate_mape(y_test_c, y_pred_c)
            print(f"Censored=0 RMSE: {rmse_censored:.4f}")
            print(f"Censored=0 MAPE: {mape_censored:.2f}%")
            
            importance_censored = pd.DataFrame({
                'feature': X_censored.columns,
                'importance': rf_censored.feature_importances_
            }).sort_values('importance', ascending=False)
            print("\nCensored=0 Feature Importance (Top 10):")
            print(importance_censored.head(10))
        
        # Model 2: All data with censored as a feature
        print("\nTraining Model 2 (all data with censored feature)...")
        X_all = df[features + ['censored']]
        y_all = df['final_expandEvts']
        X_train_a, X_test_a, y_train_a, y_test_a = train_test_split(X_all, y_all, test_size=0.2, random_state=42)
        
        rf_all = RandomForestRegressor(n_estimators=100, random_state=42)
        rf_all.fit(X_train_a, y_train_a)
        y_pred_a = rf_all.predict(X_test_a)
        rmse_all = np.sqrt(mean_squared_error(y_test_a, y_pred_a))
        mape_all = calculate_mape(y_test_a, y_pred_a)
        print(f"All Data RMSE: {rmse_all:.4f}")
        print(f"All Data MAPE: {mape_all:.2f}%")
        
        importance_all = pd.DataFrame({
            'feature': X_all.columns,
            'importance': rf_all.feature_importances_
        }).sort_values('importance', ascending=False)
        print("\nAll Data Feature Importance (Top 10):")
        print(importance_all.head(10))
        
        # Plotting
        if save_plots:
            try:
                os.makedirs(output_dir, exist_ok=True)
                
                # Scatter plot for censored=0 model
                if not df_censored.empty:
                    plt.figure(figsize=(8, 6))
                    plt.scatter(y_test_c, y_pred_c, alpha=0.7)
                    plt.plot([y_test_c.min(), y_test_c.max()], [y_test_c.min(), y_test_c.max()], 'r--')
                    plt.xlabel('Actual final_expandEvts')
                    plt.ylabel('Predicted final_expandEvts')
                    plt.title('Censored=0: Predicted vs Actual')
                    plt.tight_layout()
                    scatter_path = os.path.join(output_dir, 'scatter_plot_censored.png')
                    plt.savefig(scatter_path)
                    plt.close()
                    print(f"Saved scatter plot to {scatter_path}")
                
                # Feature importance plot for censored=0 model
                if not df_censored.empty:
                    plt.figure(figsize=(8, 6))
                    plt.barh(importance_censored['feature'].head(10), importance_censored['importance'].head(10))
                    plt.xlabel('Feature Importance')
                    plt.title('Top 10 Features (Censored=0)')
                    plt.gca().invert_yaxis()
                    plt.tight_layout()
                    importance_path = os.path.join(output_dir, 'feature_importance_censored.png')
                    plt.savefig(importance_path)
                    plt.close()
                    print(f"Saved feature importance plot to {importance_path}")
            except Exception as e:
                print(f"Error saving plots: {e}")
        
        return {
            'rmse_censored': rmse_censored,
            'mape_censored': mape_censored,
            'feature_importance_censored': importance_censored,
            'y_test_censored': y_test_c,
            'y_pred_censored': y_pred_c,
            'rmse_all': rmse_all,
            'mape_all': mape_all,
            'feature_importance_all': importance_all,
            'y_test_all': y_test_a,
            'y_pred_all': y_pred_a
        }
    
    except Exception as e:
        print(f"Error in train_random_forest_model: {e}")
        return None

results = train_random_forest_model(df)
results

Imputing missing log features...
Selected features: ['num_stackdepth3_logs', 'evts_1', 'expandEvts_1', 'pruneBacktrackEvts_1', 'backtrackEvts_1', 'strengthenEvts_1', 'maxStackDepth_1', 'evts_2', 'expandEvts_2', 'pruneBacktrackEvts_2', 'backtrackEvts_2', 'strengthenEvts_2', 'maxStackDepth_2', 'evts_3', 'expandEvts_3', 'pruneBacktrackEvts_3', 'backtrackEvts_3', 'strengthenEvts_3', 'maxStackDepth_3', 'censored', 'avg_evts', 'max_evts', 'avg_expandEvts', 'max_expandEvts', 'avg_pruneBacktrackEvts', 'max_pruneBacktrackEvts', 'evts_4', 'expandEvts_4', 'pruneBacktrackEvts_4', 'backtrackEvts_4', 'strengthenEvts_4', 'maxStackDepth_4', 'evts_5', 'expandEvts_5', 'pruneBacktrackEvts_5', 'backtrackEvts_5', 'strengthenEvts_5', 'maxStackDepth_5', 'n', 'k', 'total_sum', 'variance', 'skewness', 'max_num', 'min_num', 'avg_subset_sum', 'max_to_avg_ratio', 'range_to_avg_ratio', 'coef_of_variation', 'expandEvts_ratio_1', 'pruneBacktrackEvts_ratio_1', 'expandEvts_ratio_2', 'pruneBacktrackEvts_ratio_2', 'expa

{'rmse_censored': 66681103.768287495,
 'mape_censored': 22.85608929180363,
 'feature_importance_censored':                        feature  importance
 24      avg_pruneBacktrackEvts    0.278332
 20                    avg_evts    0.168544
 22              avg_expandEvts    0.149607
 25      max_pruneBacktrackEvts    0.126962
 21                    max_evts    0.072612
 15        pruneBacktrackEvts_3    0.052167
 23              max_expandEvts    0.028396
 13                      evts_3    0.022514
 14                expandEvts_3    0.020111
 8                 expandEvts_2    0.013025
 16             backtrackEvts_3    0.009244
 9         pruneBacktrackEvts_2    0.007125
 7                       evts_2    0.004047
 46            max_to_avg_ratio    0.004035
 44                     min_num    0.003589
 11            strengthenEvts_2    0.003429
 12             maxStackDepth_2    0.002876
 26                      evts_4    0.002688
 54  pruneBacktrackEvts_ratio_3    0.002061
 38           